This is an implementation of the artistic neural network as described in this [paper](https://arxiv.org/abs/1508.06576). VGG pretrained model weights can be downloaded from [here](http://www.vlfeat.org/matconvnet/models/). Use imagenet-vgg-verydeep-19.mat and imagenet-vgg-verydeep-16.mat

In [3]:
import os
import sys
import numpy as np
import scipy.io
import scipy.misc
import tensorflow as tf

In [134]:
class VGG(object):
    
    def __init__(self):
        self.graph = None
        self.input_shape = [1, 600, 800, 3]
    
    def factory(name, model_path):
        """
        The factory to create the corresponding model we will use
        Available names include "VGG16" and "VGG19"
        """
        if name == "VGG16": return VGG16(model_path)
        if name == "VGG19": return VGG19(model_path)
        
    factory = staticmethod(factory)
    
    def load_model(self):
        raise NotImplementedError("Abstract method")
    
    def content_loss(self):
        raise NotImplementedError("Abstract method")
    
    def style_loss(self):
        raise NotImplementedError("Abstract method")
        
    def emit(self):
        return self.graph
    
    def resize_input(new_shape):
        self.input_shape = new_shape
        tf.reshape(self.graph["input"], new_shape)

In [4]:
class VGG19(VGG):
    
    def __init__(self, model_path):
        self.path  = model_path
        self.model = load_model()
    
    def load_model(self):
        
        vgg_layers = scipy.io.loadmat(self.path)['layers']
        num_layers = len(vgg_layers[0])
        
        graph = {}
        graph["input"] = tf.Variable(np.zeros(self.input_shape), dtype=tf.float32)
        prev = "input"
        for idx in range(num_layers):
            

In [122]:
vgg = scipy.io.loadmat("models/imagenet-vgg-verydeep-16.mat")

In [123]:
w =  vgg['layers'][0]       [0]      [0]      [0]      [2]                                    [0]      [0] ## weight
b =  vgg['layers'][0]       [0]      [0]      [0]      [2]                                    [0]      [1] ## bias
                #  always 0 |layer idx|always 0|always 0|0:layer name; 1:layer type; 2: weights|always 0|0:weight; 1:bias
    
W = tf.constant(w)
print W

Tensor("Const_3:0", shape=(3, 3, 3, 64), dtype=float32)


In [133]:
print len(vgg['layers'][0])
print vgg['layers'][0][30][0][0][0][0]

37
pool5
